# Assignment 6, GR 10

In this assignment three different cases of recrystallization of grains:
<ul>
    <li> Johnson-Mehl </li>
    <li> Site-saturation </li>
    <li>Site-saturation with varying growth rate, r </sup>
    
</ul>

The growth rate is given by
$$ G(r,t) = M*P_{D} $$
where P<sub>D</sub> is stored deformation energy and
$$ M = M_{0}*exp(-Q/RT) $$

<ul>
<li> Q - activation energy [J/mol] </li>
<li> R - the universal gas constant [J/(mol*K)]</li>
<li> T - temperature in Kelvin [K]
</ul>

The kinetics equation (1) for all cases is given on the following form:
$$ X(t) = 1 - exp(-kt^{n}) $$
where X(t) is fraction of recrystallized grains, t is time (s) and k and n are constants determined by the different cases.


To define the time interval fitting the cases, it is necessary to calculate the time when 50% of the grains are recrystallized. This is done by solving equation (1) for t when X is equal to 0.5:
$$ t_{0.5} = ( \frac{ln 2}{k})^{1/n} $$


The Avrami plot consists of ln(t) plotted against ln(ln(1/(1-X(t))

In [3]:
# import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display
from IPython.html.widgets import *
from IPython.display import clear_output

newparams = {'figure.figsize': (15, 7), 'axes.grid': False,
             'lines.markersize': 10, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral', 'figure.dpi': 200}
plt.rcParams.update(newparams)

#Constants
PD = 1*10**(5) #[Pa]
M0 = 1*10**4 #[m/s*Pa]
Q = 156000 #[J/mol]
T = 300+273 #[K]
N = 2*10**12 #[m^-3]
dNdt = 1.6*10**14 #[nuclei/m^3]
R = 8.31 #[J/molK]



## Johnson-Mehl case

For this case, n is equal to 4 and X(t) is equal to
$$  X(t) = 1 - exp(-\frac{1}{3}\pi\frac{dN}{dt}G^{3}t^{4}) $$
From the equation, one sees that k is equal to the constants before t.

In [4]:
def JM(T):
    
    """
    This is the function for Johnson-Mehl. The function takes in the temperature and returns the plot for recrystalization
    fraction vs time, as well as the avrami plot.
    """
    M = M0*np.exp(-Q/(R*T))
    G= M*PD
    
    #finding the time and recrystallization values
    kJM =1/3*np.pi*dNdt*G**3
    t05 = (np.log(2)/kJM)**(1/4) #time when 50% of the grains are recrystallized 
    dt = t05/100  #defining the dt as adjustable relative to variations in kinetics.
    tid = []
    JM1 = []
    t = 0
    x=0
    while x < 0.99:
        t += dt
        x = 1-np.exp(-1/3*np.pi*dNdt*G**3*t**4)
        tid.append(t)
        JM1.append(x)
        
    
    plt.plot(tid, JM1)
    plt.xlabel("Time (s)")
    plt.ylabel("Recrystallization")
    plt.title("Johnson & Mehl case")
    plt.grid()
    plt.show()
    
    #avrami
    lntJM = []
    avJM = []
    for i in range(1,len(tid)):
        lntJM.append(np.log(tid[i]))
        avJM.append(np.log(np.log(1/(1-JM1[i]))))

    plt.plot(lntJM, avJM)
    plt.title("Avrami plot for Johnson & Mehl case")
    plt.xlabel("ln(t)")
    plt.ylabel("ln(ln(1/(1-x)))")
    plt.grid()
    plt.show()
    
    D_JM=round((G/dNdt)**(1/4)*10**6, 2)
    print(f"The grain size for the Johnson Mehl case is {D_JM} µm")

#widgets.interact(JM, T=widgets.IntSlider(min=273,max=2000,step=50,value=578, description = 'Choose temperature [K].'))
 

## Site-saturation

For this case, n is equal to 3 and X(t) is equal to
$$  X(t) = 1 - exp(-\frac{4}{3}\pi N G^{3}t^{3}) $$
From the equation, one sees that k is equal to the constants before t.

In [5]:
def SS(T):
    """
    This is the function for Site-saturation. The function takes in the temperature and returns the plot for recrystalization
    fraction vs time, as well as the avrami plot.
    """
    
    M = M0*np.exp(-Q/(R*T))
    G= M*PD
    
    kSS =4/3*np.pi*N*G**3
    t05 = (np.log(2)/kSS)**(1/3)
    dt = t05/100
    tid = []
    SS1 = []
    x=0
    t = 0
    while x < 0.99:
        t += dt
        x = 1-np.exp(-1/3*np.pi*dNdt*G**3*t**4)
        tid.append(t)
        SS1.append(x)
    
    #avrami
    lntSS = []
    avSS = []

    for i in range(1,len(tid)):
        lntSS.append(np.log(tid[i]))
        avSS.append(np.log(np.log(1/(1-SS1[i]))))
        
    plt.plot(tid, SS1)
    plt.xlabel("Time (s)")
    plt.ylabel("Recrystallization")
    plt.title("Site-saturation")
    plt.grid()
    plt.show()
    plt.plot(lntSS, avSS)
    plt.title("Avrami plot for Site-saturation case")
    plt.xlabel("ln(t)")
    plt.ylabel("ln(ln(1/(1-x)))")
    plt.grid()
    plt.show()
     

    D_SS=round((1/N)**(1/3)*10**6, 2)
    print(f"The grain size for the Site-saturation case is {D_SS} µm")
#widgets.interact(t05SS, T=widgets.IntSlider(min=273,max=2000,step=50,value=578, description = 'Choose temperature [K].'))    

## Site-saturation with varying growth rate

For this case, n is equal to 3 and X(t) is equal to
$$  X(t) = 1 - exp(-\frac{4}{3}\pi N G^{3}\frac{t^{3(1-r)}}{(1-r)^{3}}) $$
From the equation, one sees that k is equal to the constants before t.

The growth rate r is a parameter that can vary from 0-0.6.

In [6]:
def SSV(T,r):
    """
    This is the function for Site-saturation with varying growth rate. The function takes in the temperature and
    growth rate, and returns the plot for recrystalization fraction vs time, as well as the avrami plot.
    """
    
    M = M0*np.exp(-Q/(R*T))
    G= M*PD
    kSSV =4/3*np.pi*N*G**3/((1-r)**3)
    t05 = (np.log(2)/kSSV)**(1/(3*(1-r)))
    dt = t05/100 
    tid = []
    SSV1 = []
    t = 0
    x=0
    while x < 0.99:
        t += dt
        x = 1-np.exp(-4/3*np.pi*N*G**3*t**(3*(1-r))/(1-r)**3)
        tid.append(t)
        SSV1.append(x)
    
    #avrami
    lntSSV = []
    avSSV = []
    
    for i in range(1,len(tid)):
        lntSSV.append(np.log(tid[i]))
        avSSV.append(np.log(np.log(1/(1-SSV1[i]))))
    

    return tid, SSV1, lntSSV, avSSV


### Plotting the interactive menu

In [7]:
def sub_task_chooser(sub):
    
    # Switch function
    if sub == 1: #plotting for Johnson-Mehl
        widgets.interact(JM, T=widgets.IntSlider(min=273,max=1273,step=50,value=578, description = 'Temperature [K]?'))
 
        
    elif sub == 2: #plotting for site-saturation
         widgets.interact(SS, T=widgets.IntSlider(min=273,max=1273,step=50,value=578, description = 'Temperature [K]?'))
        
    elif sub == 3: #plotting for site-saturation with varying r
        rliste = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
        
        D_SSV=round((1/N)**(1/3)*10**6, 2)
        print(f"The grain size for the Site-saturation with varying growth rate case is {D_SSV} µm")
        
        for r in rliste: #making a graph for each growth rate.
            tid, SSV1, lntSSV, avSSV = SSV(T,r)
            plt.subplot(121)
            plt.plot(tid, SSV1, label = f"r = {r} ")
            plt.xlabel("Time (s)")
            plt.ylabel("Recrystallization")
            plt.title("Site-saturation with varying growth rate")
            plt.grid()
            plt.legend()
            plt.rcParams["figure.figsize"]=15,10
        
        
            plt.subplot(122)
            plt.plot(lntSSV, avSSV, label=f"r = {r} ")
            plt.title("Avrami plots for site-saturation with varying growth rate")
            plt.xlabel("ln(t)")
            plt.ylabel("ln(ln(1/(1-x)))")
            plt.legend()
            plt.grid()
        
        plt.show()
        
    
# Menu toggle buttons
sub_menu = widgets.ToggleButtons(
    #        {'names':corresponding values,}
    options={'Johnson & Mehl Case':1, 'Site-saturation':2, 'Site-saturation with varying growth rate':3},  
    value = 1,  #default value
    description='Model:',  #name
    button_style='danger',  #red buttons
    )

# T-slider
T_slider = widgets.IntSlider(
    value=573,
    min=273,
    max=1271,
    step=5,
    description='Temperature [K]',
    disabled=False,
    continuous_update=False,
    readout=True,
    readout_format='d'
)
    
widgets.interact(sub_task_chooser, sub=sub_menu, T = T_slider)

interactive(children=(ToggleButtons(button_style='danger', description='Model:', options={'Johnson & Mehl Case…

<function __main__.sub_task_chooser(sub)>